In [29]:
# demo v0.4 날짜를 입력받아 자동으로 크롤링
import os
os.chdir(r"C:\Users\cjy89\NLP\Project_news_crawling\Naver")

from bs4 import BeautifulSoup
import time
import datetime
import pandas as pd
import requests
import re

# 섹션 별 아이디
Section_ID = {"Political":"100", "Economic":"101", "Society":"102", "Culture":"103", "International":"104", "Science":"105" }

# 크롤링 시간 측정
start = time.time()
now = datetime.datetime.now()
date = "".join(str(now.date()).split('-'))

# 서버에서 뉴스 데이터 가져오기
page = 0
News_DataFrame = pd.DataFrame()

flag = True
while(flag):
    page += 1
    url = "https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&listType=title&sid1=" + Section_ID["Political"] + "&date=" + date + "&page=" + str(page)
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"}
    resp = requests.get(url, headers=headers)
    soup = BeautifulSoup(resp.text, "html.parser")

    news_box = soup.find_all(class_ = "type02")
    for box in range(len(news_box)):
        news = [i.get_text() for i in news_box[box].find_all(class_="nclicks(fls.list)")]
        urls = [i['href'] for i in news_box[box].find_all(class_="nclicks(fls.list)")]
        press = [i.get_text() for i in news_box[box].find_all(class_="writing")]
        upload = [ str(now.date()) + ' ' + str(now.hour) + ':' + str(now.minute - int(i.get_text()[:-2])) 
                  for i in news_box[box].find_all(class_="date is_new") if i.get_text() not in '시간']

        if len(upload) != 5:
            flag = False
            news = news[:len(upload)]
            urls = urls[:len(upload)]
            press = press[:len(upload)]

        temp_df = pd.DataFrame({"News":news, "Url":urls, "Press":press, "Time":upload})
        News_DataFrame = pd.concat([News_DataFrame, temp_df], axis=0, ignore_index=True)
        News_DataFrame.dropna(axis=0, inplace=True)

end = time.time()
print("Crawling Time :", end-start)

Crawling Time : 0.3711056709289551


In [30]:
News_DataFrame

,News,Url,Press,Time
0,[영상구성] 4·7 보궐선거 경선 진출자 확정,https://news.naver.com/main/read.nhn?mode=LSD&...,연합뉴스TV,2021-02-05 21:4
1,"바이든 정부 첫 한미 방위비 협상…""조속 타결"" 공감대",https://news.naver.com/main/read.nhn?mode=LSD&...,MBN,2021-02-05 21:3
2,"文대통령, 5대 해상풍력 강국 비전…""가슴 뛰는 프로젝트""(종합2보)",https://news.naver.com/main/read.nhn?mode=LSD&...,뉴시스,2021-02-05 21:2
3,"외교협회·숲나학교 임대차 분쟁…정의용 ""중재 돕겠다""",https://news.naver.com/main/read.nhn?mode=LSD&...,연합뉴스,2021-02-05 21:1
4,"문대통령 ""신안 해상풍력 사업…완전히 가슴뛰는 프로젝트""",https://news.naver.com/main/read.nhn?mode=LSD&...,디지털타임스,2021-02-05 21:-3
5,"여 “난폭운전 처벌, 길들이기 아니다”",https://news.naver.com/main/read.nhn?mode=LSD&...,경향신문,2021-02-05 21:-4
6,야 “법복만 걸친 장사꾼의 탄핵거래”,https://news.naver.com/main/read.nhn?mode=LSD&...,경향신문,2021-02-05 21:-4
7,"""김학의 출금 제보자는 공익신고자""…법무부 난감",https://news.naver.com/main/read.nhn?mode=LSD&...,SBS,2021-02-05 21:-5
8,[자료사진]영등포동 기계공구 상가 밀집지역 찾은 채현일 구청장,https://news.naver.com/main/read.nhn?mode=LSD&...,뉴시스,2021-02-05 21:-7
9,“대선 전 판 흔들어야”…‘11월 총파업’ 의결한 민노총,https://news.naver.com/main/read.nhn?mode=LSD&...,국민일보,2021-02-05 21:-10


In [ ]:
# 자동화 연습

import time
import schedule

def Thanos():
    print("I am inevitable")

## 아래 코드는 Thanos 함수를 

# 1초마다 == 매 초마다
schedule.every().second.do(Thanos)
# ** 2초 이상인 경우 seconds 를 사용해야한다.(s 붙여서) **
# 2분마다 
schedule.every(2).minutes.do(Thanos)
# 매 시간마다
schedule.every().hour.do(Thanos)
# 매일 (이렇게 하면 몇시에 실행되는걸까?)
schedule.every().day.do(Thanos)
# 매주 일요일
schedule.every().sunday.do(Thanos)
# 매일 오전 9시 49분 마다
schedule.every().day.at("09:49").do(Thanos)
# 매주 화요일 오전 6시 30분 마다
schedule.every().tuesday.at("06:30").do(Thanos)

## 실행시킨다.

## 스케줄 체크 간격은 time.sleep(몇초) 로 설정가능
while True:
    schedule.run_pending()
    time.sleep(1)